In [ ]:
!pip install --upgrade --user google-cloud-aiplatform langchain langchain-google-vertexai docarray langchain_core langchain_chroma

In [2]:
from google.colab import auth

auth.authenticate_user()

In [3]:
PROJECT_ID = "kai-ai-f63c8"
LOCATION = "us-central1"

import vertexai

vertexai.init(project = PROJECT_ID, location = LOCATION)

# Without Langchain

### Model Invoking

In [ ]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel
)

model = GenerativeModel("gemini-1.0-pro")

response = model.generate_content("Why is the sky blue?")

print(type(response))
print(f"{response}\n\n")

def output_parser(response):
  return response.candidates[0].content.parts[0].text

output_parser(response)

### Prompting Model

In [ ]:
prompt = """Create a nmbered list of 10 items. Each item in the list should be a trend in the tech industry."""

list_response = model.generate_content(prompt, stream=True)

for stream in list_response:
  print(stream.text, end="")

## Top 10 Tech Industry Trends:

1. **Artificial Intelligence (AI) Everywhere:** AI is rapidly permeating all aspects of our lives, from personalized recommendations and chatbots to self-driving cars and medical diagnostics. This trend shows no signs of slowing down, with further advancements expected in areas like natural language processing and computer vision.
2. **The Rise of the Metaverse:** The metaverse concept, encompassing immersive virtual worlds and augmented reality experiences, is gaining significant traction. Tech giants like Meta are investing heavily in this area, aiming to create a new frontier for social interaction, entertainment, and even work.
3. **Focus on Cybersecurity:** As technology advances, so does the sophistication of cyber threats. This makes robust cybersecurity a top priority for businesses and individuals alike. We can expect to see further development in areas like data encryption, threat detection, and user awareness initiatives.
4. **Quantum Computi

### Chat Prompts

In [ ]:
chat = model.start_chat(response_validation=False)

prompt = """
  My name is Mikhail. You are my personal assistant. My favorite movies are Lord of The Rings and the Hobbit

  Suggest another movie I might like
"""

responses = chat.send_message(prompt, stream=True)

for response in responses:
  print(response.text, end="")

Hello Mikhail, 

It's a pleasure to meet you. As your personal assistant, I am here to help you with anything you need. 

Since you enjoy "The Lord of the Rings" and "The Hobbit," I suggest you watch "The Chronicles of Narnia." This fantasy film series is based on the popular books by C.S. Lewis and shares many similarities with your favorite movies. 

Both franchises feature epic adventures in fantastical worlds filled with mythical creatures, heroic characters, and epic battles between good and evil. If you enjoyed the immersive storytelling and world-building in "The Lord of the Rings" and "The Hobbit," you'll likely appreciate the same qualities in "The Chronicles of Narnia."

Here are some additional reasons why you might enjoy "The Chronicles of Narnia":

* **Rich mythology and lore:** Like Tolkien's Middle-earth, Narnia is a richly detailed world with its own unique history, mythology, and creatures. 
* **Engaging characters:** The Chronicles of Narnia features a diverse cast of

In [ ]:
prompt = "Are my favorite based on a book series?"

responses = chat.send_message(prompt, stream=True)

for response in responses:
  print(response.text, end="")

Yes, both "The Lord of the Rings" and "The Hobbit" are based on book series written by J.R.R. Tolkien. 

"The Lord of the Rings" is a trilogy of epic high fantasy novels consisting of "The Fellowship of the Ring," "The Two Towers," and "The Return of the King." The story follows a group of hobbits, elves, dwarves, and men on a quest to destroy the One Ring, an artifact created by the evil lord Sauron. 

"The Hobbit," published in 1937, serves as a prequel to "The Lord of the Rings" and tells the story of Bilbo Baggins, a hobbit who embarks on an adventure to reclaim a treasure guarded by the dragon Smaug. 

Both book series have been immensely popular and have been translated into numerous languages. They have also been adapted into multiple films, television shows, video games, and other media. 

If you enjoyed the movies, I highly recommend reading the books. They provide a much deeper and richer experience, allowing you to fully immerse yourself in Tolkien's incredible world and cha

In [ ]:
print(chat.history)

[role: "user"
parts {
  text: "\n  My name is Mikhail. You are my personal assistant. My favorite movies are Lord of The Rings and the Hobbit\n\n  Suggest another movie I might like\n"
}
, role: "model"
parts {
  text: "Hello Mikhail, \n\nIt\'s a pleasure to meet you. As your personal assistant, I am here to help you with anything you need. \n\nSince you enjoy \"The Lord of the Rings\" and \"The Hobbit,\" I suggest you watch \"The Chronicles of Narnia.\" This fantasy film series is based on the popular books by C.S. Lewis and shares many similarities with your favorite movies. \n\nBoth franchises feature epic adventures in fantastical worlds filled with mythical creatures, heroic characters, and epic battles between good and evil. If you enjoyed the immersive storytelling and world-building in \"The Lord of the Rings\" and \"The Hobbit,\" you\'ll likely appreciate the same qualities in \"The Chronicles of Narnia.\"\n\nHere are some additional reasons why you might enjoy \"The Chronicle

# With Langchain

### Model Invoking

In [12]:
from langchain_google_vertexai import VertexAI as langchain_VertexAI

google_llm = langchain_VertexAI(model_name="gemini-pro", temperature=0.2, max_tokens=1000)
# For a specific version: model = VertexAI(model_name="gemini-1.0-pro-002")

message = "What are some of the pros and cons of python as a programming langauge?"

# Invoke model normally
# google_llm.invoke(message)

# You can also return safety attributes
# result = google_llm.generate([message])

# Support streaming responses
for chunk in google_llm.stream(message):
  print(chunk, end="", flush=True)

## Pros of Python:

* **Easy to learn and read:** Python has a simple and intuitive syntax, making it a great choice for beginners and experienced programmers alike. Its code resembles natural language, making it easier to understand and maintain.
* **Versatile:** Python can be used for a wide range of tasks, including web development, data analysis, machine learning, scripting, and more. This versatility makes it a valuable tool for many different types of projects.
* **Large and active community:** Python has a large and active community of developers, which means there are plenty of resources available to help you learn and solve problems. You can easily find libraries, frameworks, and tutorials online.
* **Open-source and free:** Python is an open-source language, which means it is free to use and distribute. This makes it a great choice for individuals and organizations who are looking for a cost-effective solution.
* **Extensive libraries and frameworks:** Python has a vast ecosy

In [9]:
google_llm_code = langchain_VertexAI(model_name = "code-bison", max_tokens = 1000, temperature=0.3)

question = "Write a python function that checks if a string is a valid email address"

print(google_llm_code.invoke(question))

```python
import re

def is_valid_email(email):
  """
  Checks if a string is a valid email address.

  Args:
    email: The string to check.

  Returns:
    True if the string is a valid email address, False otherwise.
  """

  # Compile the regular expression.
  regex = re.compile(r"[^@]+@[^@]+\.[^@]+")

  # Check if the string matches the regular expression.
  return regex.match(email) is not None
```


### Vertex Model Garden
We can use any model available with an endpoint from vertex ai. If you have an endpoint deployed with google, you can call that model such as Llama 3, Anthropic Claude, etc.

### Prompt Templates

In [16]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

print(prompt.format(product="colorful socks"))
print(prompt.format(product="red shoes"))
print(prompt.format(product="books"))

chain = prompt | google_llm
chain.invoke({"product": "Headphones"})

What is a good name for a company that makes colorful socks?
What is a good name for a company that makes red shoes?
What is a good name for a company that makes books?


'## Catchy Names for a Headphone Company:\n\n**Descriptive:**\n\n* **Harmonic Haven**\n* **Sound Sanctuary**\n* **Audio Oasis**\n* **Clarity Acoustics**\n* **Immersive Soundscapes**\n\n**Creative:**\n\n* **Earphoria**\n* **Headspace**\n* **Mindwave**\n* **Soundscape**\n* **Sonic Bloom**\n\n**Unique:**\n\n* **Audify**\n* **Hearo**\n* **Musephonic**\n* **Rhythmix**\n* **Sonify**\n\n**Modern:**\n\n* **Amplify**\n* **Beatbox**\n* **Decibel**\n* **Frequency**\n* **Headset**\n\n**Playful:**\n\n* **Earworm**\n* **Headbanger**\n* **Melody Maker**\n* **Rhythm & Rhyme**\n* **Soundbyte**\n\n**Bonus:**\n\n* **[Your Name] Audio**\n* **[Location] Sounds**\n* **[Unique Adjective] Headphones**\n\n**Tips for Choosing a Name:**\n\n* **Keep it short and memorable.**\n* **Make it easy to pronounce and spell.**\n* **Consider the target audience.**\n* **Choose a name that reflects the brand identity.**\n* **Check for availability of the domain name and trademark.**\n\n**I hope this helps you find the perfec

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template)
])

chain = chat_prompt | google_llm
chain.invoke({"text": "Hello! My name is Mikhail", "input_language": "English", "output_language": "Spanish"})

' ¡Hola! Mi nombre es Mikhail'

### Output Parsers
Use output parsers to have more fine-grained control over your model!

#### JSON Output Parsing

In [19]:
question_topic = "Create a multiple choice question on large language models"

prompt = PromptTemplate(
    template="Answer the user query. Respond in JSON {topic}\n",
    input_variables=["topic"]
)

chain = prompt | google_llm
chain.invoke({"topic": question_topic})

'```json\n{\n  "question": "Which of the following is NOT a characteristic of large language models (LLMs)?",\n  "choices": [\n    "They can generate human-quality text.",\n    "They are trained on massive amounts of data.",\n    "They can understand and respond to complex questions.",\n    "They are capable of independent thought and creativity."\n  ],\n  "correctAnswer": "They are capable of independent thought and creativity."\n}\n```'

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class Quiz_Question(BaseModel):
  question: str = Field(description="question for the quiz")
  choices: List[str] = Field(description="A list of string choices to the question")
  answer: str = Field(description="answer to the question")

quiz_query = "Make a multiple choice quiz question on Large Language Models"

parser = JsonOutputParser(pydantic_object = Quiz_Question)

prompt = PromptTemplate(
    template="Answer the user query. \n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | google_llm | parser

chain.invoke({"query": quiz_query})

{'question': 'What is a Large Language Model (LLM)?',
 'choices': ['A type of artificial intelligence that can understand and generate human language.',
  'A computer program that can translate languages.',
  'A search engine that can answer questions in natural language.',
  'A virtual assistant that can help with tasks like scheduling appointments and setting reminders.'],
 'answer': 'A type of artificial intelligence that can understand and generate human language.'}

### Chaining

Shoutout to James Briggs for an amazing in-depth explanation of the Langchain Runnable and LCEL

https://www.youtube.com/watch?v=O0dUOtOIrfs

In [50]:
def add_five(x: int):
  return x + 5

def multiply_by_two(x: int):
  return x * 2

multiply_by_two(add_five(3))

16

In [51]:
# func | other | other2 | other3
class CustomRunnable():
  def __init__(self, func):
    self.func = func

  def __or__(self, other):
    def chained_func(*args, **kwargs):
      # result of previous function is input for this function
      return other(self.func(*args, **kwargs))
    return CustomRunnable(chained_func)

  def __call__(self, *args, **kwargs):
    return self.func(*args, **kwargs)

In [52]:
# Wrap functions with CustomRunnable
add_five = CustomRunnable(add_five)
multiply_by_two = CustomRunnable(multiply_by_two)

# Run using object approach
chain = add_five.__or__(multiply_by_two)
chain(3)

16

In [54]:
chain = add_five | multiply_by_two
chain(3)

16

### Runnables

In [ ]:
!pip install langchain-chroma

In [106]:
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_chroma import Chroma

embedding = VertexAIEmbeddings(model = "textembedding-gecko")

vectorstore_A = Chroma.from_texts(
    [
        "Mikhail's birthday is the 7th of March",
        "Mikhail works as an AI engineer and specializes in LLMs and LLMOps",
    ],
    embedding = embedding
)
vectorstore_B = Chroma.from_texts(
    [
        "Alice has a small yorkshire shitzu mix who is 3 years old",
        "Alice was born in 1996",
    ],
    embedding = embedding
)

In [107]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import PromptTemplate

retriever_A = vectorstore_A.as_retriever()
retriever_B = vectorstore_B.as_retriever()

prompt = """
  Answer the question below using the context. Respond with I don't know when context does not support answering the question

  Context: {context}

  Question: {question}

  Answer: """
prompt = PromptTemplate.from_template(prompt)

retrieval = RunnableParallel(
    {"context": retriever_A, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | google_llm

In [112]:
out = chain.invoke("What does Mikhail do that Alice doesn't?")
print(out)

I don't know. The context provided does not mention Alice or what she does. 



In [110]:
prompt = """
  Answer the question below using the context. Respond with I don't know when context does not support answering the question

  Context A: {context_a}
  Context B: {context_b}

  Question: {question}

  Answer: """
prompt = PromptTemplate.from_template(prompt)

retrieval = RunnableParallel(
    {"context_a": retriever_A, "context_b": retriever_B, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | google_llm

In [113]:
out = chain.invoke("What does Mikhail do?")
print(out)

Mikhail works as an AI engineer and specializes in LLMs and LLMOps.
